In [1]:
from data_gen import *
from models import *
from keras import models

Using TensorFlow backend.


In [2]:
# data generator
dg = DataGen(data="data/wolayitta_clean.txt")

# length of a word
n_input_length = len(char2int)
n_steps_in = dg.max_root_len
n_steps_out = dg.max_output_len
print(n_steps_in, n_steps_out)

15 31


In [3]:
print("Total train data: ", len(dg.words) * .7)
batch_size = 128
# number of batches to train
n_batches = int(len(dg.words) * .7 / batch_size) 

# python generator to generate training data at each request
# E.x word_matrix, feature = next(gen)
gen = dg.cnn_gen_data(batch_size=batch_size, n_batches=n_batches)

Total train data:  579040.0


In [4]:
# infenc - inference encoder model
# infdec - inference decoder model
# train - training model that combines both
# n_input_length - the length of the input and the output
# word_feat_len - the length of the word feature vector
# n_units - size of the hidden memory in the RNN
train, infenc, infdec = conv_model(n_input_length, n_input_length, dg.word_feat_len, 256)
train.compile(optimizer='adam', loss='categorical_crossentropy')


In [5]:
train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
root_word_input (InputLayer)    (None, 15, 28, 1)    0                                            
__________________________________________________________________________________________________
cnn (Conv2D)                    (None, 15, 28, 20)   520         root_word_input[0][0]            
__________________________________________________________________________________________________
pooling (MaxPooling2D)          (None, 5, 9, 20)     0           cnn[0][0]                        
__________________________________________________________________________________________________
flatten (Flatten)               (None, 900)          0           pooling[0][0]                    
__________________________________________________________________________________________________
word_featu

In [ ]:
model = models.load_model("model.h5")
root_input = model.get_layer("root_word_input").output
feature_input = model.get_layer("word_feature_input").output
target_input = model.get_layer("target_word_input").output

state_h = model.get_layer("concatnate").output
encoder_model = Model([root_input, feature_input], state_h)


decoder_state_input_h = Input(shape=(256,))
decoder_gru = model.get_layer("decoder_gru")
gru_outputs, state_h  = decoder_gru(target_input, initial_state=decoder_state_input_h)

decoder_dense = model.get_layer("train_output")
decoder_outputs = decoder_dense(gru_outputs)

decoder_model = Model([target_input, decoder_state_input_h], [decoder_outputs, state_h])

In [ ]:
# model train given the data generator, how many batches and number of epochs
# history = train.fit_generator(gen, steps_per_epoch=n_batches, epochs = 2)

In [ ]:

# data generator for gofa word, root word, word feature
dg2 = DataGen(data="data/goffa.txt")


# length of a word
# n_input_length = len(char2int)
# n_steps_in = dg2.max_root_len
# if n_steps_out >= dg2.max_output_len:
    

print(n_steps_in, n_steps_out)

In [ ]:
# number of batches for the gofa data generator
n_batches = int(len(dg2.words) * .1 / batch_size) 
gen2 = dg2.cnn_gen_data(76batch_size=batch_size, n_batches=n_batches)
print("Train size: ", (n_batches * batch_size))

In [ ]:
n_batches * 128

In [ ]:
# transfer the learned weights of wolaytta encoder from above
# fix it not to be trainable, only the encoder
encoder_model.trainable = False

# train the decoder only with gofa data
history = model.fit_generator(gen2, steps_per_epoch=n_batches, epochs = 6)

In [ ]:
history.history['loss']

In [ ]:
# test data generator for gofa words

g_test_batches = int(len(dg2.words) * .03 / batch_size) 
gen3 = dg2.cnn_gen_data(batch_size=batch_size, n_batches=g_test_batches, trainset=False)
print("Train size: ", (g_test_batches * batch_size))

In [ ]:

# shows sample examples and calculates accuracy

total, correct = 0, 0
in_word = 0
sims = []
for b in range(3):
    # get data from test data generator
    [X1, X2, X3], y = next(gen3)
    for j in range(batch_size):
        word_features = X3[j].reshape((1, X3.shape[1])) 
        root_word_matrix = X1[j].reshape((1, X1.shape[1], X1.shape[2], 1))
        
        # predicts the target word given root word and features
        target = predict(infenc, infdec, root_word_matrix, word_features, n_steps_out, n_input_length)
        root = ''.join(dg.one_hot_decode(X1[j]))#.replace('&', ' ')
        word = ''.join(dg.one_hot_decode(y[j]))#.replace('&', ' ')
        targetS = ''.join(dg.one_hot_decode(target))#.replace('&', ' ')
#         sims.append(dg.word_sim(word, targetS))
        
        # checks if the predicted and the real words are equal'
#         print(len(dg.one_hot_decode(y[j])), len(dg.one_hot_decode(target)))
#         print(len(dg.one_hot_decode(target)[:27]))
#         print(len(y[j]))
        if dg.one_hot_decode(y[j]) == dg.one_hot_decode(target)[:27]:
            correct += 1
#         else:
#             print(root, word.split('&')[0], '\t\t', targetS.split('&')[0])
        if root.strip() in targetS.strip():
            in_word += 1
#     print(b, root, word, targetS)
    total += batch_size
    

print('Exact Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))